A few months ago, I inherited a code base devoid of any tests. I realised quite
quickly there might be a better design for the entire project, and wanted to get
there cautiously via continuous refactoring. 

Without the support harness of a test suite, it is hard to guarantee refactored
code has the same functionality as the old, battle-tested code. As such, refactoring
legacy projects begins with writing a test suite covering most (or all) execution
paths. For my project I did this by hand, but that laborious process piqued my
interest in automated test generation techniques.

I recently came across [this](https://www.fuzzingbook.org/beta/html/PrototypingWithPython.html)
fascinating piece by Andreas Zeller on automated testing techniques in Python.
Zeller prototypes a symbolic test generator based around the `z3` SMT solver. His
prototype is already impressive, but quite limited. We extend the symbolic approach,
and also include concrete values from earlier executions, resulting in a simple
concolic test generator.
<!-- TEASER_END -->

Let us first import the essentials.

In [1]:
import ast
import inspect

import astor
import z3

Zeller takes the following, well-known _triangle_ function that classifies a
triangle with integral sides `a`, `b`, and `c` into one of three categories (type
hints mine).

In [2]:
def triangle(a: int, b: int, c: int) -> str:
    if a == b:
        if b == c:
            return 'equilateral'
        else:
            return 'isosceles #1'
    else:
        if b == c:
            return 'isosceles #2'
        else:
            if a == c:
                return 'isosceles #3'
            else:
                return 'scalene'

In a few lines of code, he then develops a simple symbolic test generator that 
systematically creates test inputs covering all possible execution paths in the
triangle function. The following cells reproduce the meat of his approach.

In [3]:
triangle_source = inspect.getsource(triangle)
triangle_ast = ast.parse(triangle_source)

In [4]:
def collect_path_conditions(tree):
    paths = []

    def traverse_if_children(children, context, cond):
        old_paths = len(paths)

        for child in children:
            traverse(child, context + [cond])

        if len(paths) == old_paths:
            paths.append(context + [cond])

    def traverse(node, context):
        if isinstance(node, ast.If):
            cond = astor.to_source(node.test).strip()
            not_cond = "z3.Not" + cond

            traverse_if_children(node.body, context, cond)
            traverse_if_children(node.orelse, context, not_cond)
        else:
            for child in ast.iter_child_nodes(node):
                traverse(child, context)

    traverse(tree, [])

    return ["z3.And(" + ", ".join(path) + ")" for path in paths]

In [5]:
path_conditions = collect_path_conditions(triangle_ast)
path_conditions

['z3.And((a == b), (b == c))',
 'z3.And((a == b), z3.Not(b == c))',
 'z3.And(z3.Not(a == b), (b == c))',
 'z3.And(z3.Not(a == b), z3.Not(b == c), (a == c))',
 'z3.And(z3.Not(a == b), z3.Not(b == c), z3.Not(a == c))']

In [6]:
a = z3.Int('a')
b = z3.Int('b')
c = z3.Int('c')

s = z3.Solver()

In [7]:
for path_condition in path_conditions:
    s = z3.Solver()
    s.add(a > 0, b > 0, c > 0)
    eval(f"s.check({path_condition})")
    m = s.model()
    print(m, triangle(m[a].as_long(), m[b].as_long(), m[c].as_long()))

[b = 1, a = 1, c = 1] equilateral
[b = 1, a = 1, c = 2] isosceles #1
[b = 2, a = 1, c = 2] isosceles #2
[b = 2, a = 1, c = 1] isosceles #3
[b = 2, a = 1, c = 3] scalene


From the output, it is clear that a simple function like _triangle_ can easily
be tested using a symbolic test generator. 

The limitation, of course, is that most functions in realistic programmes are
nowhere near as simple. In particular, symbolic testing any and all functions
in a programme requires a full symbolic interpreter for the language. Not all
Python constructs might be supported by the `z3` theories, but we can at least
expand the current prototype to include many other common constructs.

# Expanding the symbolic interpreter

TODO

## Looping constructs

TODO

## Function calls

TODO

## Flow control 2.0: returns and exceptions

TODO